In [1]:
#Basic Imports
import pandas as pd
import warnings
import time
warnings.filterwarnings('ignore')

# Web Scraping and parsing
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument('--log-level=3')

prefs = {"download.default_directory" : "/home/guest/dbms/co_author"}
options.add_experimental_option("prefs",prefs)

# Create new automated instance of Chrome
driver = webdriver.Chrome(executable_path="chromedriver", chrome_options=options)

In [3]:
# Get the faculty names
page_links = ['https://iitg.ac.in/dsai/faculty', 'https://iitg.ac.in/dsai/assoc_faculty_list']
            #    'https://iitg.ac.in/dsai/distinguished_faculty']
names = []
# Get the faculty table
for link in ['https://iitg.ac.in/dsai/head']:
    driver.get(link)
    table = driver.find_element(By.XPATH, '//*[@id="head_table"]')
    name = " ".join(table.find_element(By.XPATH, f'//*[@id="head_table"]/tbody/tr[1]/td[2]').text.split("\n")[1].split(".")[1:]).lstrip().strip()
    names.append(name)

for link in page_links:
    driver.get(link)
    table = driver.find_element(By.XPATH, '//*[@id="assoc_fac_table"]')
    for row in range(len(table.find_elements(By.XPATH, '//*[@id="assoc_fac_table"]/tbody/tr'))):
        name = " ".join(table.find_element(By.XPATH, f'//*[@id="assoc_fac_table"]/tbody/tr[{row+1}]/td[2]').text.split("\n")[0].split(".")[1:]).lstrip().strip()
        names.append(name)



In [4]:
# Reaching the faculty irins page
faculty_links = []
for name in names:
    link = "https://iitg.irins.org/"
    driver.get(link)

    # Find the search box and enter the researcher's name
    search_box = driver.find_element(By.XPATH, '//*[@id="title"]')
    search_box.send_keys(name)

    # Submit the search query
    search_box.submit()

    # Wait for the search results to load
    driver.implicitly_wait(10)

    exact_search = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[3]/div/div/form/div/div[1]/div/div/div[2]/span[2]/input')
    exact_search.click()

    search = driver.find_element(By.XPATH, '//*[@id="search"]')
    search.click()

    # Wait for the search results to load
    driver.implicitly_wait(10)

    # Click on the first search result to go to the researcher's page
    faculty_link = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[3]/div/div/form/div/div[3]/div/div[3]/div/div/div[2]/div/a').get_attribute('href')

    # Wait for the researcher's page to load
    driver.implicitly_wait(10)

    faculty_links.append(faculty_link)


In [59]:
len(faculty_links) == len(names)

True

In [72]:
SCROLL_PAUSE_TIME = 0.5

for idx, link in enumerate(faculty_links):
    driver.get(link)
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    DOIs = [ele.text for ele in driver.find_elements(By.XPATH, '//*[@title="Doi Link"]')]
    titles = [ele.text for ele in driver.find_elements(By.XPATH, '//*[@class="row"]/h2')]
    data = dict(zip(titles, DOIs))
    DOIs_data = pd.DataFrame.from_dict(data, orient = 'index').reset_index()
    DOIs_data.columns = ['Titles', 'DOIs']
    name = "_".join(names[idx].split(" "))
    DOIs_data.to_csv(f'DOI_data/{name}.csv', index = False)

In [11]:
import os
for idx, link in enumerate(faculty_links):
    id = link.split("/")[-1]
    name = "_".join(names[idx].split(" "))
    driver.get(f"https://iitg.irins.org/assets/files/coauthor_{id}.csv")
    # Source file path
    source = f'/home/guest/dbms/co_author/coauthor_{id}.csv'
    # destination file path
    dest = f'/home/guest/dbms/co_author/{name}.csv'
    os.rename(source, dest)


FileNotFoundError: [Errno 2] No such file or directory: '/home/guest/dbms/co_author/coauthor_128315.csv' -> '/home/guest/dbms/co_author/Ratnajit_Bhattacharjee.csv'